[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/leafmap/blob/master/examples/notebooks/57_national_map.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://gishub.org/leafmap-binder)

### Downloading various shapes from the National Map

The national map (TNM) is a catalog of *topological* datasources maintained by the USGS. 

- It contains a wide range of dataformats (such as GeoTiff, LAZ, ...) and datasets.
- It provides an endpoint that can be used to search for published datasets and files.
- This API supports a wide range of searchable parameters (bounding box, polygon, dates, keyword, ...)
- It returns detailed information regarding the properties of datasets, file,
- as well as various download links (file, thumbnail, xml descriptions, ...).

We've created a thin wrapper to expose this treasure trove. 

- For more details about TNM, see https://apps.nationalmap.gov/tnmaccess/#/
- The same data is also downloable using https://apps.nationalmap.gov/downloader/

In [1]:
import leafmap

### Usage

A class groups the functionalities together.

In [2]:
TNM = leafmap.The_national_map_USGS()

### Datasets

In [3]:
TNM.datasets

{'Alaska IFSAR 5 meter DEM',
 'Digital Elevation Model (DEM) 1 meter',
 'Ifsar Digital Surface Model (DSM)',
 'Ifsar Orthorectified Radar Image (ORI)',
 'Lidar Point Cloud (LPC)',
 'National Elevation Dataset (NED) 1 arc-second',
 'National Elevation Dataset (NED) 1/3 arc-second',
 'National Elevation Dataset (NED) 1/3 arc-second - Contours',
 'National Elevation Dataset (NED) 1/9 arc-second',
 'National Elevation Dataset (NED) Alaska 2 arc-second',
 'National Hydrography Dataset (NHD) Best Resolution',
 'National Hydrography Dataset Plus High Resolution (NHDPlus HR)',
 'National Watershed Boundary Dataset (WBD)',
 'Original Product Resolution (OPR) Digital Elevation Model (DEM)',
 'Small-scale Datasets - Boundaries',
 'Small-scale Datasets - Contours',
 'Small-scale Datasets - Hydrography',
 'Small-scale Datasets - Transportation',
 'Topobathymetric Lidar DEM',
 'Topobathymetric Lidar Point Cloud',
 'US Topo Current',
 'US Topo Historical'}

### Formats

Note that any format (f.e. 'All') is specific to one or more datasets.

In [4]:
TNM.prodFormats

{'All',
 'FileGDB',
 'GeoPDF',
 'GeoPackage',
 'GeoTIFF',
 'GeoTIFF, IMG',
 'JPEG2000',
 'LAS,LAZ',
 'Shapefile',
 'TIFF',
 'TXT (pipes)'}

### Looking for files

In [5]:
TNM.find_details().keys(), TNM.find_details()['total']

(dict_keys(['total', 'items', 'errors', 'messages', 'sciencebaseQuery', 'filteredOut']),
 11190958)

### A detail

In [6]:
TNM.find_details()['items'][0]

{'title': '18TWK610820',
 'moreInfo': 'Lidar (Light detection and ranging) discrete-return point cloud data are available in the American Society for Photogrammetry and Remote Sensing (ASPRS) LAS format. The LAS format is a standardized binary format for storing 3-dimensional point cloud data and point attributes along with header information and variable length records specific to the data. Millions of data points are stored as a 3-dimensional data cloud as a series of x (longitude), y (latitude) and z (elevation) points. A few older projects in this collection are in ASCII format. Please refer to http://www.asprs.org/Committee-General/LASer-LAS-File-Format-Exchange-Activities.html for additional information. This data set is a LAZ (compressed LAS) format file containing [...]',
 'sourceId': '6338596ad34e900e86cdbfd3',
 'sourceName': 'ScienceBase',
 'sourceOriginId': None,
 'sourceOriginName': 'gda',
 'metaUrl': 'https://www.sciencebase.gov/catalog/item/6338596ad34e900e86cdbfd3',
 've

### Using parameters

In [7]:
params = {
    'q': 'National Elevation Dataset (NED) 1/3 arc-second',
    'polyCode': '01010002',
    'polyType': 'huc8',
}

TNM.find_details(**params)['total']

25

In [8]:
params = {
    'prodFormats': 'LAS,LAZ',
    'datasets': 'Lidar Point Cloud (LPC)',
    'polygon': [
        (-104.94262695312236, 41.52867510196275),
        (-102.83325195312291, 40.45065268246805),
        (-104.94262695312236, 40.45065268246805),
        (-104.94262695312236, 41.52867510196275),
    ],
}

TNM.find_details(**params)['total']

13837

Available parameters

In [9]:
help(TNM.find_details)

Help on method find_details in module leafmap.common:

find_details(bbox: List[float] = None, polygon: List[Tuple[float, float]] = None, datasets: str = None, prodFormats: str = None, prodExtents: str = None, q: str = None, dateType: str = None, start: str = None, end: str = None, offset: int = 0, max: int = None, outputFormat: str = 'JSON', polyType: str = None, polyCode: str = None, extentQuery: int = None) -> Dict method of leafmap.common.The_national_map_USGS instance
    Possible search parameters (kwargs) support by API
    
    Parameter               Values
        Description
    ---------------------------------------------------------------------------------------------------
    bbox                    'minx, miny, maxx, maxy'
        Geographic longitude/latitude values expressed in  decimal degrees in a comma-delimited list.
    polygon                 '[x,y x,y x,y x,y x,y]'
        Polygon, longitude/latitude values expressed in decimal degrees in a space-delimited list

### Max items

Defaults to about 50. You only retrieve about 1000 items in one call.

In [10]:
len(TNM.find_details()['items'])

50

In [11]:
len(TNM.find_details(max=1000000)['items'])

998

Use offset to retrieve more batches.

In [12]:
params = {
    'q': 'National Elevation Dataset (NED) 1/3 arc-second',
    'polyCode': '01010002',
    'polyType': 'huc8',
    'max': 2,
}

TNM.find_details(**params, offset=0)['items'][0] == TNM.find_details(
    **params, offset=1
)['items'][0]

False

### Select a region from leafmap

In [13]:
m = leafmap.Map(center=[40, -100], zoom=4)
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [14]:
region = m.user_roi_bounds()
if region is None:
    region = [-115.9689, 35.9758, -115.3619, 36.4721]

In [15]:
TNM.find_details(q='LAZ', bbox=region)['total']

313

### Error handling

In [16]:
bool(TNM.find_details(start='01-01-2010', q='NED', bbox=region))

Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


False

In [17]:
bool(TNM.find_details(start='2021-12-01', end='2020-01-01', q='NED', bbox=region))

Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


False

In [18]:
bool(TNM.find_details(start='2021-12-01', end='2022-01-01', q='NED', bbox=region))

Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


False

In [19]:
bool(
    TNM.find_details(
        start='2020-12-01',
        end='2022-01-01',
        q='NED',
        dateType='dateCreated',
        bbox=region,
    )
)

True

### Downloading files 

In [20]:
help(TNM.download_tiles)

Help on method download_tiles in module leafmap.common:

download_tiles(region=None, out_dir=None, download_args={}, geopandas_args={}, API={}) -> None method of leafmap.common.The_national_map_USGS instance
    Download the US National Elevation Datasets (NED) for a region.
    
    Args:
        region (str | list, optional): An URL|filepath to a vector dataset Or a list of bounds in the form of [minx, miny, maxx, maxy].
            Alternatively you could use API parameters such as polygon or bbox.
        out_dir (str, optional): The directory to download the files to. Defaults to None, which uses the current working directory.
        download_args (dict, optional): A dictionary of arguments to pass to the download_file function. Defaults to {}.
        geopandas_args (dict, optional): A dictionary of arguments to pass to the geopandas.read_file() function.
            Used for reading a region URL|filepath.
        API (dict, optional): A dictionary of arguments to pass to the se

In [21]:
params = {
    'q': 'National Elevation Dataset (NED) 1/3 arc-second',
    'polyCode': '01010002',
    'polyType': 'huc8',
    'max': 0,
}

TNM.download_tiles(API=params)

Downloading...
From: https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n47w069/USGS_1_n47w069_20210611.tif
To: /media/hdd/Dropbox/git/leafmap/examples/notebooks/USGS_1_n47w069_20210611.tif
 50%|████▉     | 27.3M/54.9M [00:07<00:07, 3.73MB/s]

Cancelled download
0 Downloads completed, 0 downloads failed, 25 files available


It can also be accessed without invoking the class.

In [22]:
params = {
    'q': 'National Elevation Dataset (NED) 1/3 arc-second',
    'polyCode': '01010002',
    'polyType': 'huc8',
    'max': 0,
}

leafmap.download_tnm(API=params)

Downloading...
From: https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/historical/n47w069/USGS_1_n47w069_20210611.tif
To: /media/hdd/Dropbox/git/leafmap/examples/notebooks/USGS_1_n47w069_20210611.tif
 16%|█▌        | 8.91M/54.9M [00:02<00:11, 3.88MB/s]

Cancelled download
0 Downloads completed, 0 downloads failed, 25 files available


In [ ]:
region = [-115.9689, 35.9758, -115.3619, 36.4721]

leafmap.download_ned(region=region, return_url=True) == leafmap.download_tnm(
    region=region, return_url=True, API={'q': 'NED'}
)

### List of files

In [ ]:
TNM.find_tiles(API=params)

### Dataset metadata

In [ ]:
TNM.datasets_full[0]

### Read the docs

In [ ]:
help(TNM)